In [40]:
import duckdb

MinIO Setup

In [ ]:
minioAuth = {
    'accessKey': 'minioadmin',
    'secretKey': 'miniopassword',
    'bucket': 'data-dapodik-kemendikdasmen',
    'endpoint': 'http://localhost:9000'
}

Path yang Digunakan

In [42]:
localCatalogFile = "../metadata/dapodikMetadata.ducklake"
minioNewDataFolder = f"s3://{minioAuth['bucket']}/ducklake/JawaBarat/" # Folder tempat DuckLake menyimpan data barunya
currentMiniofilePath = f"s3://{minioAuth['bucket']}/JawaBarat/dataDKIJakarta.parquet" # LOKASI FILE PARQUET LIVE ANDA
print(currentMiniofilePath)

s3://data-dapodik-kemendikdasmen/JawaBarat/dataDKIJakarta.parquet


DuckDB Connection & Install Extension 

In [43]:
con = duckdb.connect()
con.sql("INSTALL httpfs; LOAD httpfs;")
con.sql("INSTALL ducklake; LOAD ducklake;")

Set MinIO Credentials

In [44]:
con.sql(f"SET s3_endpoint = '{minioAuth['endpoint'].replace('http://', '')}';")
con.sql("SET s3_use_ssl = false;") 
con.sql(f"SET s3_access_key_id = '{minioAuth['accessKey']}';")
con.sql(f"SET s3_secret_access_key = '{minioAuth['secretKey']}';")
con.sql("SET s3_url_style = 'path';")

ATTACH ke DuckLake

In [45]:
con.sql(f"ATTACH 'ducklake:{localCatalogFile}' AS jakartaLake (DATA_PATH '{currentMiniofilePath}');")
con.sql("USE jakartaLake;")

print("✅ DuckLake sudah ter-ATTACH dan siap digunakan.")

✅ DuckLake sudah ter-ATTACH dan siap digunakan.


### Langkah 2: Memuat Data Live ke DuckLake (Metadata Management)
Kita akan menggunakan perintah CREATE TABLE AS SELECT untuk membaca file Parquet live Anda dan menuliskannya ke Tabel DuckLake baru.

Perintah ini akan melakukan dua hal:
1. Membuat Schema
1. Membaca konten dari MINIO_LIVE_PARQUET (Parquet yang sudah ada).
2. Menulis data tersebut dalam format yang dikelola DuckLake (dengan commit log metadata) ke dalam Data Path DuckLake Anda (MINIO_DATA_PATH).

In [46]:
## -- 1. Buat Schema --
con.sql("CREATE SCHEMA IF NOT EXISTS staging;")

# --- 2. Daftarkan dan Load Data Live Anda ---
# Anda bisa melakukan TRANSFORMASI di sini jika diperlukan, atau hanya SELECT *
# staging -> skema, dataDKIJakarta -> tabel
loadLiveDataQuery = f"""
    CREATE TABLE staging.dataDKIJakarta AS
    SELECT 
        * -- Ambil semua kolom dari file Parquet live Anda
    FROM 
        read_parquet('{currentMiniofilePath}');
"""


con.sql(loadLiveDataQuery)
print(f"\n✅ Data Parquet Live dari {currentMiniofilePath} berhasil dimuat dan terdaftar sebagai 'curated.data_pangan_managed' di DuckLake.")
print(f"File data fisik telah disalin dan sekarang dikelola di folder: {minioNewDataFolder}")

# --- 3. Verifikasi Snapshot Awal ---
print("\n--- Riwayat Snapshot Awal ---")
con.sql("SELECT * FROM ducklake_snapshots('jakartaLake', 'staging.dataDKIJakarta');").df()

InvalidInputException: Invalid Input Error: No magic bytes found at end of file 's3://data-dapodik-kemendikdasmen/JawaBarat/dataDKIJakarta.parquet'

LINE 6:         read_parquet('s3://data-dapodik-kemendikdasmen/JawaBarat...
                ^